# Optimised Neural Network

In this notebook, we will look into common problem in neural networks, which is overfitting. We will use the a sin(x) and popular MNIST dataset for this purpose, and use different architectures and different train test splits to see how our model performs. Then we try to find optimal hyperparameters for our model to get the best performance.

# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.
# MATIN read the links in the end.

## Introduction

Overfitting is a common problem in neural networks, where the model performs well on the training data but poorly on the test data. This is because the model has learned the noise in the training data, rather than the underlying pattern. Common reasons for overfitting include using a model that is too complex, not having enough training data, and training for too many epochs. There are several ways to reduce overfitting, such as using dropout, early stopping, and data augmentation. In this notebook, we will explore some of these techniques and see how they can be used to improve the performance of a neural network.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy, sparse_categorical_crossentropy
from tensorflow.keras import backend as K
from keras.models import load_model
from keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split
import imageio.v3 as iio
import os
import shutil
from glob import glob
from PIL import Image
from tqdm import tqdm
from pygifsicle import optimize

In [2]:
files_layers = glob("../data/OptimisedNN/sin/layers/*")
files_ratios = glob("../data/OptimisedNN/sin/ratios/*")

for file in files_layers:
    shutil.rmtree(file)
for file in files_ratios:
    shutil.rmtree(file)

## Dataset

At first we will use a simple sin(x) dataset to demonstrate overfitting. We will generate 1000 data points between 0 and 4*pi and use this as our training data. We will then use different architectures and train test splits to see how our model performs. We will then use the popular MNIST dataset to further demonstrate overfitting and explore different techniques to reduce it.

## Regression 


# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.
# MATIN the analysis and report is not complete.

We first analyze overfitting on a simple regression problem. 

In [3]:
x_sin_values = np.linspace(0, 4 * np.pi, 1000)
y_sin_values = np.sin(x_sin_values)

At first we try with 2-layer neural network with different ratios of train test splits, and see how it performs.

In [4]:
TRAIN_TEST_RATIOS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
EPOCHS = [100, 200, 500]
LOSS_FUNCTIONS = ["mse", "mae"]

Hyperparameters are things like learning rate, batch size, number of epochs, etc. that are set before the training process begins. These are the parameters that are set before the training process begins, and are not learned by the model. Hyperparameters can have a significant impact on the performance of a model, and finding the right hyperparameters can be a challenging task. In this notebook, we will explore some common hyperparameters and see how they can be tuned to improve the performance of a neural network.

Some common hyperparameters include:

- Learning rate: The rate at which the model learns from the training data. A higher learning rate can help the model converge faster, but may result in overshooting the optimal solution. A lower learning rate can help the model converge more slowly, but may result in getting stuck in local minima.
- Batch size: The number of samples that are used to update the model parameters in each iteration. A larger batch size can help the model converge faster, but may require more memory. A smaller batch size can help the model converge more slowly, but may result in more noise in the updates.
- Number of epochs: The number of times the model sees the entire training data. A larger number of epochs can help the model learn more from the training data, but may result in overfitting. A smaller number of epochs can help the model generalize better, but may result in underfitting.
- Activation function: The function that is applied to the output of each neuron. Common activation functions include `ReLU`, sigmoid, and tanh. The choice of activation function can have a significant impact on the performance of the model.
- Number of layers: The number of layers in the neural network. A deeper network can learn more complex patterns, but may require more data and more training time. A shallower network may be easier to train, but may not be able to learn as complex patterns.
- Number of neurons: The number of neurons in each layer of the neural network. A larger number of neurons can help the model learn more complex patterns, but may require more data and more training time. A smaller number of neurons may be easier to train, but may not be able to learn as complex patterns.
- Dropout rate: The rate at which neurons are randomly dropped out during training. Dropout can help prevent overfitting by reducing the co-adaptation of neurons, but may require more training time.
- Optimizer: The algorithm used to update the model parameters based on the loss function. Common optimizers include SGD, Adam, and RMSprop. The choice of optimizer can have a significant impact on the performance of the model.
- Weight initialization: The technique used to initialize the weights of the neural network. Common weight initialization techniques include random initialization, Xavier initialization, and He initialization. The choice of weight initialization can have a significant impact on the performance of the model.
- Learning rate schedule: The technique used to adjust the learning rate during training. Common learning rate schedules include step decay, exponential decay, and cosine annealing. The choice of learning rate schedule can have a significant impact on the performance of the model.
- Regularization Parameter: The parameter used to control the strength of the regularization term. A larger regularization parameter can help prevent overfitting, but may result in underfitting. A smaller regularization parameter may not be able to prevent overfitting, but may result in better performance on the training data.

In [5]:
def create_model(num_layers, num_neurons, activation, loss):
    model = Sequential()
    model.add(Input(shape=(1,)))
    for i in range(num_layers):
        model.add(Dense(num_neurons, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss=loss)
    return model

In [6]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [7]:
def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))

In [8]:
def mean_absolute_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))

In [9]:
def error_stats(mses, rmses, maes, y_true, y_pred, ratio=None, num_layer=None):
    mse = mean_squared_error(y_true, y_pred)
    mses.append(mse)
    
    rmse = root_mean_squared_error(y_true, y_pred)
    rmses.append(rmse)
    
    mae = mean_absolute_error(y_true, y_pred)
    maes.append(mae)
    
    print("For " + (f"ratio {ratio}" if ratio else f"layer {num_layer}"))
    print(f"Mean Squared Error: {mse}")
    print(f"Root Mean Squared Error: {rmse}")
    print(f"Mean Absolute Error: {mae}")
    print()

In [10]:
FILE_PATH = "../data/OptimisedNN/sin/"

In [11]:
def plot_sin_predictions_ratio(loss, epochs, activation, num_neurons):
    accuracy_mses = []
    accuracy_rmses = []
    accuracy_maes = []
    
    model_path = (
        FILE_PATH
        + "ratios/"
        + f"model_layers.{2}_neurons.{num_neurons}_epochs.{epochs}_activation.{activation}_loss.{loss}/"
    )
    
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    for i, ratio in enumerate(TRAIN_TEST_RATIOS):
        model = create_model(2, num_neurons, activation, loss)
        file_path = model_path + f"ratio.{ratio}/"

        if not os.path.exists(file_path):
            os.makedirs(file_path)

        x_train, x_test, y_train, y_test = train_test_split(x_sin_values, y_sin_values, test_size=ratio)
        

        history = model.fit(x_train, y_train, epochs=epochs, verbose=0)

        y_test_pred = model.predict(x_test)

        error_stats(accuracy_mses, accuracy_rmses, accuracy_maes, y_test, y_test_pred, ratio=ratio)

        plt.figure()
        plt.plot(history.history["loss"])
        plt.title(f"Loss function changes for ratio {ratio}")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        loss_filename = f"{file_path}loss_{i}.png"
        plt.savefig(loss_filename)
        plt.close()

        y_pred = model.predict(x_sin_values)
        plt.figure()
        plt.plot(x_sin_values, y_sin_values, label="True function")
        plt.plot(x_sin_values, y_pred, label="Estimated function")
        plt.title(f"Function estimation for ratio {ratio}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        sin_filename = f"{file_path}sin_{i}.png"
        plt.savefig(sin_filename)
        plt.close()

    frames_loss = np.stack(
        [
            iio.imread(f"{model_path}ratio.{TRAIN_TEST_RATIOS[i]}/loss_{i}.png")
            for i in range(len(TRAIN_TEST_RATIOS))
        ]
    )
    iio.imwrite(f"{model_path}loss.gif", frames_loss, fps=0.5)

    frames_sin = np.stack(
        [
            iio.imread(f"{model_path}ratio.{TRAIN_TEST_RATIOS[i]}/sin_{i}.png")
            for i in range(len(TRAIN_TEST_RATIOS))
        ]
    )
    iio.imwrite(f"{model_path}sin.gif", frames_sin, fps=0.5)

    plt.figure()
    plt.plot(TRAIN_TEST_RATIOS, accuracy_mses, label="Mean Squared Error")
    plt.plot(TRAIN_TEST_RATIOS, accuracy_rmses, label="Root Mean Squared Error")
    plt.plot(TRAIN_TEST_RATIOS, accuracy_maes, label="Mean Absolute Error")
    plt.title("Error changes for different ratios")
    plt.xlabel("Train test ratio")
    plt.ylabel("Error")
    plt.legend(loc="upper right")
    plt.savefig(f"{model_path}errors.png")
    plt.close()

    # for removing files
    # os.remove(loss_filename)
    # os.remove(sin_filename)

At first we try with MSE loss and 100 epochs and relu, and see how it performs.

In [12]:
plot_sin_predictions_ratio(loss="mse", epochs=100, activation="relu", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
For ratio 0.1
Mean Squared Error: 0.9995403501126733
Root Mean Squared Error: 0.999770148640513
Mean Absolute Error: 0.8102009021073969

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
For ratio 0.2
Mean Squared Error: 0.972737436119481
Root Mean Squared Error: 0.9862745237100475
Mean Absolute Error: 0.8016031009324875

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
For ratio 0.3
Mean Squared Error: 0.8945123512560306
Root Mean Squared Error: 0.9457866309353451
Mean Absolute Error: 0.7718300980828083

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
For ratio 0.4
Mean Squared Error: 0.8900276661332928
Root Mean Squared Error: 0.9434127761130293
Mean Absolute Error: 0.7702103805225946

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.5
Mean Squared Error: 0.8746587855352449
Root Mean Squared Error: 0.9352319421059382


Now we try with 200 epochs and see how it performs.

In [13]:
plot_sin_predictions_ratio(loss="mse", epochs=200, activation="relu", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
For ratio 0.1
Mean Squared Error: 0.9047098313102786
Root Mean Squared Error: 0.9511623580179561
Mean Absolute Error: 0.7731208126032766

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
For ratio 0.2
Mean Squared Error: 0.954609976750788
Root Mean Squared Error: 0.9770414406517197
Mean Absolute Error: 0.7928650315434146

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
For ratio 0.3
Mean Squared Error: 0.934795193920955
Root Mean Squared Error: 0.9668480717884042
Mean Absolute Error: 0.7861933266747307

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.4
Mean Squared Error: 0.9215227015525179
Root Mean Squared Error: 0.9599597395477156
Mean Absolute Error: 0.7849118998087934

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 0.9501596132661484
Root Mean Squared Error: 0.974761310919

Now we try with 500 epochs and see how it performs.

In [14]:
plot_sin_predictions_ratio(loss="mse", epochs=500, activation="relu", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
For ratio 0.1
Mean Squared Error: 1.0165122171054486
Root Mean Squared Error: 1.0082223053996815
Mean Absolute Error: 0.8161786773536535

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
For ratio 0.2
Mean Squared Error: 0.9698468175735881
Root Mean Squared Error: 0.9848080105145308
Mean Absolute Error: 0.7994303946373708

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.3
Mean Squared Error: 0.9660836789912803
Root Mean Squared Error: 0.9828955585367554
Mean Absolute Error: 0.7979092972176925

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.4
Mean Squared Error: 0.9911368305404503
Root Mean Squared Error: 0.9955585520402355
Mean Absolute Error: 0.8071725506784944

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.5
Mean Squared Error: 0.9728653941058228
Root Mean Squared Error: 0.9863393909

Now with different loss functions and see how it performs.

In [15]:
plot_sin_predictions_ratio(loss="mae", epochs=100, activation="relu", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
For ratio 0.1
Mean Squared Error: 1.0433200451532332
Root Mean Squared Error: 1.021430391731729
Mean Absolute Error: 0.8257449384771275

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
For ratio 0.2
Mean Squared Error: 0.7687306490790797
Root Mean Squared Error: 0.8767728605967909
Mean Absolute Error: 0.7242903271224793

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
For ratio 0.3
Mean Squared Error: 0.8937845310272563
Root Mean Squared Error: 0.9454017828559751
Mean Absolute Error: 0.7720363466287442

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
For ratio 0.4
Mean Squared Error: 0.8593328407287582
Root Mean Squared Error: 0.9270020715881697
Mean Absolute Error: 0.7599701389918774

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
For ratio 0.5
Mean Squared Error: 0.8464229811731323
Root Mean Squared Error: 0.92001248968322

In [16]:
plot_sin_predictions_ratio(loss="mae", epochs=200, activation="relu", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
For ratio 0.1
Mean Squared Error: 1.12156299709413
Root Mean Squared Error: 1.0590387136899813
Mean Absolute Error: 0.8573444878739341

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
For ratio 0.2
Mean Squared Error: 1.0081235080042557
Root Mean Squared Error: 1.0040535384152858
Mean Absolute Error: 0.8158504389704169

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.3
Mean Squared Error: 1.0523967003098587
Root Mean Squared Error: 1.0258638800103348
Mean Absolute Error: 0.8344142116255402

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.4
Mean Squared Error: 1.0360835606016021
Root Mean Squared Error: 1.0178818991423328
Mean Absolute Error: 0.8259491629048465

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 1.0355952851619394
Root Mean Squared Error: 1.017642022108

In [17]:
plot_sin_predictions_ratio(loss="mae", epochs=500, activation="relu", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
For ratio 0.1
Mean Squared Error: 1.0434785028138474
Root Mean Squared Error: 1.0215079553355653
Mean Absolute Error: 0.8230648700085909

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
For ratio 0.2
Mean Squared Error: 1.0237649043831203
Root Mean Squared Error: 1.0118126824581317
Mean Absolute Error: 0.8182854655802024

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 0.988786671319576
Root Mean Squared Error: 0.9943775295729363
Mean Absolute Error: 0.8047404268100552

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 1.0295679194713374
Root Mean Squared Error: 1.0146762633822364
Mean Absolute Error: 0.8218544985343275

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 1.039884451645314
Root Mean Squared Error: 1.0197472489

Now we change architecture to use `tanH` activation function and see how it performs.

In [18]:
plot_sin_predictions_ratio(loss="mse", epochs=100, activation="tanh", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
For ratio 0.1
Mean Squared Error: 0.9601371051495889
Root Mean Squared Error: 0.9798658607940114
Mean Absolute Error: 0.7955025238462389

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 0.9721314455819237
Root Mean Squared Error: 0.9859672639504435
Mean Absolute Error: 0.8000192989208814

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 0.9591478884520672
Root Mean Squared Error: 0.979360959223956
Mean Absolute Error: 0.794953325646777

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 0.898303457731608
Root Mean Squared Error: 0.9477887199854238
Mean Absolute Error: 0.7716470404433344

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 0.9086596815429733
Root Mean Squared Error: 0.95323642478

In [19]:
plot_sin_predictions_ratio(loss="mse", epochs=200, activation="tanh", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
For ratio 0.1
Mean Squared Error: 0.9898342755854097
Root Mean Squared Error: 0.9949041539693206
Mean Absolute Error: 0.802498866004221

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 0.9760647518025803
Root Mean Squared Error: 0.9879598938229124
Mean Absolute Error: 0.8020116641218001

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 0.9943711375191621
Root Mean Squared Error: 0.9971815970620206
Mean Absolute Error: 0.8078522945700031

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 1.0132672462151415
Root Mean Squared Error: 1.0066117653868056
Mean Absolute Error: 0.8136699311249833

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 0.9974289774980071
Root Mean Squared Error: 0.998713661

In [20]:
plot_sin_predictions_ratio(loss="mse", epochs=500, activation="tanh", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
For ratio 0.1
Mean Squared Error: 0.9355677376689863
Root Mean Squared Error: 0.9672475058995946
Mean Absolute Error: 0.7860356609362598

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 0.9586691944357985
Root Mean Squared Error: 0.9791165377194885
Mean Absolute Error: 0.7933972677966467

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 0.9989807532284846
Root Mean Squared Error: 0.9994902466900237
Mean Absolute Error: 0.8093901485221542

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 0.9743182645594046
Root Mean Squared Error: 0.9870756123820529
Mean Absolute Error: 0.799986701470727

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 0.9721882961864162
Root Mean Squared Error: 0.985996093

In [21]:
plot_sin_predictions_ratio(loss="mae", epochs=100, activation="tanh", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
For ratio 0.1
Mean Squared Error: 1.0163831433439834
Root Mean Squared Error: 1.008158292801276
Mean Absolute Error: 0.8178692252047989

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 1.0032280845652441
Root Mean Squared Error: 1.001612741814542
Mean Absolute Error: 0.8096088992533794

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 1.0266341237593566
Root Mean Squared Error: 1.0132295513650185
Mean Absolute Error: 0.8199684982928477

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 1.0229081466882362
Root Mean Squared Error: 1.0113892162210532
Mean Absolute Error: 0.8190662236831544

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 0.9587810240755759
Root Mean Squared Error: 0.9791736434

In [22]:
plot_sin_predictions_ratio(loss="mse", epochs=200, activation="tanh", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
For ratio 0.1
Mean Squared Error: 1.005254815175808
Root Mean Squared Error: 1.002623964991765
Mean Absolute Error: 0.8115824908259986

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 1.0380754777695524
Root Mean Squared Error: 1.0188598911379094
Mean Absolute Error: 0.822515812963892

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 1.0003056278226112
Root Mean Squared Error: 1.0001528022370438
Mean Absolute Error: 0.8087065350749999

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 1.0126993289632271
Root Mean Squared Error: 1.0063296323587154
Mean Absolute Error: 0.8129271924610595

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 0.9660500477637496
Root Mean Squared Error: 0.98287845014

In [23]:
plot_sin_predictions_ratio(loss="mse", epochs=500, activation="tanh", num_neurons=100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
For ratio 0.1
Mean Squared Error: 0.8597283463535185
Root Mean Squared Error: 0.9272153721512163
Mean Absolute Error: 0.7542218912886078

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 1.027804733957583
Root Mean Squared Error: 1.0138070496685172
Mean Absolute Error: 0.8159655164148516

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 0.8818739542682212
Root Mean Squared Error: 0.9390814417654207
Mean Absolute Error: 0.7645031837413523

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 0.9948118916245138
Root Mean Squared Error: 0.9974025724974414
Mean Absolute Error: 0.8083845758202629

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 1.0001685839161611
Root Mean Squared Error: 1.000084288

Now we repeat some steps with different number of neurons and see how it performs.

In [24]:
plot_sin_predictions_ratio(loss="mae", epochs=100, activation="relu", num_neurons=200)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
For ratio 0.1
Mean Squared Error: 1.0592490216048542
Root Mean Squared Error: 1.0291982421306667
Mean Absolute Error: 0.8312799660952589

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 0.9656492621872598
Root Mean Squared Error: 0.9826745454051712
Mean Absolute Error: 0.7986091994681949

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 1.0095991711611851
Root Mean Squared Error: 1.00478812252195
Mean Absolute Error: 0.8158796493503273

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 1.0910990888166883
Root Mean Squared Error: 1.0445568863478372
Mean Absolute Error: 0.8478566277305261

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 0.9589297629253216
Root Mean Squared Error: 0.9792495917

In [25]:
plot_sin_predictions_ratio(loss="mae", epochs=100, activation="relu", num_neurons=500)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
For ratio 0.1
Mean Squared Error: 1.0543008095942092
Root Mean Squared Error: 1.026791512233233
Mean Absolute Error: 0.8278744507241281

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 0.9806816619426849
Root Mean Squared Error: 0.9902937250849795
Mean Absolute Error: 0.8034202032653962

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 1.017950238416679
Root Mean Squared Error: 1.0089352003060845
Mean Absolute Error: 0.8165294097595907

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.4
Mean Squared Error: 1.0610589350662623
Root Mean Squared Error: 1.0300771500554036
Mean Absolute Error: 0.8314175248237748

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 1.018714115413834
Root Mean Squared Error: 1.00931368533

In [26]:
plot_sin_predictions_ratio(loss="mae", epochs=200, activation="relu", num_neurons=200)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
For ratio 0.1
Mean Squared Error: 1.0645231525524808
Root Mean Squared Error: 1.0317573128175448
Mean Absolute Error: 0.828240949328188

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
For ratio 0.2
Mean Squared Error: 1.0558373005781476
Root Mean Squared Error: 1.027539439913694
Mean Absolute Error: 0.8282026713168356

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.3
Mean Squared Error: 0.9598495994591898
Root Mean Squared Error: 0.9797191431523576
Mean Absolute Error: 0.794831564460817

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
For ratio 0.4
Mean Squared Error: 1.014905275836611
Root Mean Squared Error: 1.007425072070678
Mean Absolute Error: 0.8169949774851295

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For ratio 0.5
Mean Squared Error: 0.9657965416712664
Root Mean Squared Error: 0.9827494806263

Now we use 90% train and 10% test split and see how it performs with different number of layers.

In [27]:
NUM_LAYERS = np.linspace(2, 20, 19, dtype=int)

In [28]:
def plot_sin_predictions_layers(loss, epochs, activation, num_neurons):
    accuracy_mses = []
    accuracy_rmses = []
    accuracy_maes = []

    model_path = (
        FILE_PATH
        + "layers/"
        + f"model_epochs._neurons.{num_neurons}_epochs.{epochs}_activation.{activation}_loss.{loss}/"
    )
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    
    for i, num_layer in enumerate(NUM_LAYERS):
        model = create_model(num_layer, num_neurons, activation, loss)
        
        file_path = model_path + f"layers.{num_layer}/"
        if not os.path.exists(file_path):
            os.makedirs(file_path)        

        x_train, x_test, y_train, y_test = train_test_split(x_sin_values, y_sin_values, test_size=.9)

        history = model.fit(x_train, y_train, epochs=epochs, verbose=0)

        y_test_pred = model.predict(x_test)

        error_stats(accuracy_mses, accuracy_rmses, accuracy_maes, y_test, y_test_pred, num_layer=num_layer)

        plt.figure()
        plt.plot(history.history["loss"])
        plt.title(f"Loss function changes for layers {num_layer}")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        loss_filename = f"{file_path}loss_{i}.png"
        plt.savefig(loss_filename)
        plt.close()

        y_pred = model.predict(x_sin_values)
        plt.figure()
        plt.plot(x_sin_values, y_sin_values, label="True function")
        plt.plot(x_sin_values, y_pred, label="Estimated function")
        plt.title(f"Function estimation for layers {num_layer}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        sin_filename = f"{file_path}sin_{i}.png"
        plt.savefig(sin_filename)
        plt.close()
        
    frames_loss = np.stack(
        [
            iio.imread(f"{model_path}layers.{NUM_LAYERS[i]}/loss_{i}.png")
            for i in range(len(NUM_LAYERS))
        ]
    )
    iio.imwrite(f"{model_path}loss.gif", frames_loss, fps=0.5)
    
    frames_sin = np.stack(
        [
            iio.imread(f"{model_path}layers.{NUM_LAYERS[i]}/sin_{i}.png")
            for i in range(len(NUM_LAYERS))
        ]
    )
    iio.imwrite(f"{model_path}sin.gif", frames_sin, fps=0.5)

    plt.figure()
    plt.plot(NUM_LAYERS, accuracy_mses, label="Mean Squared Error")
    plt.plot(NUM_LAYERS, accuracy_rmses, label="Root Mean Squared Error")
    plt.plot(NUM_LAYERS, accuracy_maes, label="Mean Absolute Error")
    plt.title("Error changes for different layers")
    plt.xlabel("Number of layers")
    plt.ylabel("Error")
    plt.legend(loc="upper right")    
    plt.savefig(f"{model_path}errors.png")
    plt.close()

Now we try with Mean Squared Error loss and 100 epochs and `ReLU` activation function with 100 neurons in each layer.

In [29]:
plot_sin_predictions_layers(loss="mse", epochs=100, activation="relu", num_neurons=100)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.5835343173541042
Root Mean Squared Error: 0.7638941794215376
Mean Absolute Error: 0.664318554917115

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 3
Mean Squared Error: 0.6313656478557885
Root Mean Squared Error: 0.7945852049061752
Mean Absolute Error: 0.6814913237038683

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 636us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 4
Mean Squared Error: 0.7920922008185761
Root Mean Squared Error: 0.8899956184266168
Mean Absolute Error: 0.7385287794224256

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 0.8907548703227405
Root Mean Squared Error: 0.943798108878557
Mean Absolute Error: 0.7725437805914217

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 6
Mean Squared Error: 0.861390912941381
Root Mean Squared Error: 0.9281114765702345
M

Now we try with Mean Squared Error loss and 200 epochs and `ReLU` activation function with 100 neurons in each layer.

In [30]:
plot_sin_predictions_layers(loss="mse", epochs=200, activation="relu", num_neurons=100)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.7035496990854
Root Mean Squared Error: 0.8387786949400896
Mean Absolute Error: 0.706094758133382

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 3
Mean Squared Error: 0.7378707596947921
Root Mean Squared Error: 0.8589940393825746
Mean Absolute Error: 0.7188055649304653

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 4
Mean Squared Error: 1.019512519691705
Root Mean Squared Error: 1.0097091262792988
Mean Absolute Error: 0.8188073975083748

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 0.9353524469364997
Root Mean Squared Error: 0.9671362090918216
Mean Absolute Error: 0.7881472089123385

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 6
Mean Squared Error: 0.9858572016164616
Root Mean Squared Error: 0.9929034200849857
Mean

Now we try with Mean Squared Error loss and 500 epochs and `ReLU` activation function with 100 neurons in each layer.

In [31]:
plot_sin_predictions_layers(loss="mse", epochs=500, activation="relu", num_neurons=100)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.7094662238217051
Root Mean Squared Error: 0.8422981798755742
Mean Absolute Error: 0.7077528905610436

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 3
Mean Squared Error: 0.9341804865982307
Root Mean Squared Error: 0.9665301271032531
Mean Absolute Error: 0.7865560860411861

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 4
Mean Squared Error: 1.0006396585322757
Root Mean Squared Error: 1.0003197781371094
Mean Absolute Error: 0.8110429715277925

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 0.9931480561188974
Root Mean Squared Error: 0.9965681392252601
Mean Absolute Error: 0.8076364680125442

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 6
Mean Squared Error: 0.9313728240374163
Root Mean Squared Error: 0.9650765897261296

Now we try with Mean Squared Error loss and 100 epochs and `ReLU` activation function with 200 neurons in each layer.

In [32]:
plot_sin_predictions_layers(loss="mse", epochs=100, activation="relu", num_neurons=200)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.639705840543469
Root Mean Squared Error: 0.7998161292093758
Mean Absolute Error: 0.6822911493329451

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 3
Mean Squared Error: 0.6819468368207431
Root Mean Squared Error: 0.8258007246429027
Mean Absolute Error: 0.6988415536748366

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 4
Mean Squared Error: 0.7727325582133334
Root Mean Squared Error: 0.879052079352147
Mean Absolute Error: 0.7315770432342696

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 5
Mean Squared Error: 0.9476597624006644
Root Mean Squared Error: 0.9734781776704933
Mean Absolute Error: 0.7903206780310292

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 6
Mean Squared Error: 0.9120447902978468
Root Mean Squared Error: 0.955010361356277
Me

Now we try with Mean Squared Error loss and 100 epochs and `ReLU` activation function with 500 neurons in each layer.

In [33]:
plot_sin_predictions_layers(loss="mse", epochs=100, activation="relu", num_neurons=500)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 2
Mean Squared Error: 0.6669334028229759
Root Mean Squared Error: 0.8166599064622775
Mean Absolute Error: 0.69438949758622

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 3
Mean Squared Error: 0.6373190970563054
Root Mean Squared Error: 0.7983226772779948
Mean Absolute Error: 0.6823944508720118

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 4
Mean Squared Error: 0.8592074537478522
Root Mean Squared Error: 0.9269344387538162
Mean Absolute Error: 0.7584268658316401

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
For layer 5
Mean Squared Error: 1.0328407734860148
Root Mean Squared Error: 1.0162877414817197
Mean Absolute Error: 0.8246524734909387

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
For layer 6
Mean Squared Error: 0.7821483697980661
Root Mean Squared Error: 0.8843915251731362
Mean A

Now we try with Mean Squared Error loss and 200 epochs and `ReLU` activation function with 200 neurons in each layer.

In [34]:
plot_sin_predictions_layers(loss="mse", epochs=200, activation="relu", num_neurons=200)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.7011690289051087
Root Mean Squared Error: 0.8373583634890791
Mean Absolute Error: 0.7034779499558519

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 3
Mean Squared Error: 0.8352231595250275
Root Mean Squared Error: 0.9139054434267406
Mean Absolute Error: 0.7504043282562255

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 4
Mean Squared Error: 0.9893722176134034
Root Mean Squared Error: 0.9946719145594709
Mean Absolute Error: 0.8067146458693577

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 0.9684576962376216
Root Mean Squared Error: 0.9841024825888925
Mean Absolute Error: 0.7976679349950867

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 6
Mean Squared Error: 0.9449525588366116
Root Mean Squared Error: 0.9720867033534671


Now we try with Mean Squared Error loss and 200 epochs and `ReLU` activation function with 500 neurons in each layer.

In [35]:
plot_sin_predictions_layers(loss="mse", epochs=200, activation="tanh", num_neurons=100)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.6013204126126922
Root Mean Squared Error: 0.7754485235092605
Mean Absolute Error: 0.6702708141196706

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 3
Mean Squared Error: 0.9595760008911401
Root Mean Squared Error: 0.9795795020778764
Mean Absolute Error: 0.7950379113873851

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 4
Mean Squared Error: 0.9737927667372624
Root Mean Squared Error: 0.9868093872360875
Mean Absolute Error: 0.8001027543141086

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 5
Mean Squared Error: 1.0033772074011866
Root Mean Squared Error: 1.0016871804117224
Mean Absolute Error: 0.8118736065215184

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 6
Mean Squared Error: 1.0217388127619103
Root Mean Squared Error: 1.010810967867835

Now we try with Mean Squared Error loss and 100 epochs and `tanH` activation function with 100 neurons in each layer.

In [36]:
plot_sin_predictions_layers(loss="mse", epochs=100, activation="tanh", num_neurons=100)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.5586102330882908
Root Mean Squared Error: 0.747402323443198
Mean Absolute Error: 0.657703907601622

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 3
Mean Squared Error: 0.5916907576046735
Root Mean Squared Error: 0.7692143768837615
Mean Absolute Error: 0.6664513631508863

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 4
Mean Squared Error: 0.8683462734372318
Root Mean Squared Error: 0.93185099315139
Mean Absolute Error: 0.7616536024565196

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 0.9454430183691465
Root Mean Squared Error: 0.9723389421231398
Mean Absolute Error: 0.7901290945790794

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 6
Mean Squared Error: 1.0483562800598387
Root Mean Squared Error: 1.0238927092522139
Mea

Now we try with Mean Squared Error loss and 200 epochs and `tanH` activation function with 100 neurons in each layer.

In [37]:
plot_sin_predictions_layers(loss="mse", epochs=200, activation="tanh", num_neurons=100)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.6141852677636929
Root Mean Squared Error: 0.7836997306135131
Mean Absolute Error: 0.6766107073729337

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 3
Mean Squared Error: 0.9334937408108627
Root Mean Squared Error: 0.9661747982693725
Mean Absolute Error: 0.7847469356785943

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 4
Mean Squared Error: 0.9994813792784775
Root Mean Squared Error: 0.9997406560095861
Mean Absolute Error: 0.8095909831395426

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 0.9948581373865111
Root Mean Squared Error: 0.9974257553254333
Mean Absolute Error: 0.809264148402119

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 6
Mean Squared Error: 0.9793921489675668
Root Mean Squared Error: 0.9896424349064498
M

Now we try with Mean Squared Error loss and 500 epochs and `tanH` activation function with 100 neurons in each layer.

In [38]:
plot_sin_predictions_layers(loss="mse", epochs=500, activation="tanh", num_neurons=100)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
For layer 2
Mean Squared Error: 0.849668308576126
Root Mean Squared Error: 0.9217745432458666
Mean Absolute Error: 0.7562913866471344

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 676us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 3
Mean Squared Error: 0.9973232096789865
Root Mean Squared Error: 0.9986607079879465
Mean Absolute Error: 0.8088657934421142

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 4
Mean Squared Error: 1.0312661006041448
Root Mean Squared Error: 1.0155127279380327
Mean Absolute Error: 0.8240021711980385

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 1.0164809089562645
Root Mean Squared Error: 1.0082067788684346
Mean Absolute Error: 0.8163596345229037

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 6
Mean Squared Error: 0.9979942487333828
Root Mean Squared Error: 0.9989966209819645
M

Now we try with Mean Squared Error loss and 100 epochs and `tanH` activation function with 200 neurons in each layer.

In [39]:
plot_sin_predictions_layers(loss="mse", epochs=100, activation="tanh", num_neurons=200)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 2
Mean Squared Error: 0.6209571225244604
Root Mean Squared Error: 0.7880083264309207
Mean Absolute Error: 0.6795701882102656

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 3
Mean Squared Error: 0.6471684007589333
Root Mean Squared Error: 0.8044677748418101
Mean Absolute Error: 0.6854775258978507

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 4
Mean Squared Error: 0.9045361651353077
Root Mean Squared Error: 0.9510710620849042
Mean Absolute Error: 0.7749474771453515

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 0.9872080298578289
Root Mean Squared Error: 0.9935834287355183
Mean Absolute Error: 0.809125857956594

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
For layer 6
Mean Squared Error: 0.973015985079648
Root Mean Squared Error: 0.9864157262937611
Mean

Now we try with Mean Squared Error loss and 100 epochs and `tanh` activation function with 500 neurons in each layer.

In [40]:
plot_sin_predictions_layers(loss="mse", epochs=100, activation="tanh", num_neurons=500)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 2
Mean Squared Error: 0.5687296904799997
Root Mean Squared Error: 0.7541416912490647
Mean Absolute Error: 0.6577398490123261

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
For layer 3
Mean Squared Error: 0.6073999014306912
Root Mean Squared Error: 0.7793586474985
Mean Absolute Error: 0.6720936446616086

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
For layer 4
Mean Squared Error: 0.6122926824794108
Root Mean Squared Error: 0.782491330609746
Mean Absolute Error: 0.6732414667276513

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
For layer 5
Mean Squared Error: 0.6596790240384267
Root Mean Squared Error: 0.8122062693912345
Mean Absolute Error: 0.6899997297669287

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
For layer 6
Mean Squared Error: 0.9162841421288148
Root Mean Squared Error: 0.957227319986645
Mean Absolute 

Now we try with Mean Squared Error loss and 200 epochs and `tanH` activation function with 200 neurons in each layer.

In [41]:
plot_sin_predictions_layers(loss="mse", epochs=200, activation="tanh", num_neurons=200)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 2
Mean Squared Error: 0.681196616989953
Root Mean Squared Error: 0.8253463618323843
Mean Absolute Error: 0.6974701716496226

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
For layer 3
Mean Squared Error: 0.9560383952725715
Root Mean Squared Error: 0.9777721591825835
Mean Absolute Error: 0.7934092188884971

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 4
Mean Squared Error: 0.9235562523533475
Root Mean Squared Error: 0.9610183413199498
Mean Absolute Error: 0.7817346701222206

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For layer 5
Mean Squared Error: 0.9417893369488165
Root Mean Squared Error: 0.9704583128341044
Mean Absolute Error: 0.7882652079415247

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
For layer 6
Mean Squared Error: 0.9442973147360986
Root Mean Squared Error: 0.9717496152487525
M

Now we try with Mean Squared Error loss and 200 epochs and `tanH` activation function with 500 neurons in each layer.

In [42]:
plot_sin_predictions_layers(loss="mse", epochs=200, activation="tanh", num_neurons=500)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
For layer 2
Mean Squared Error: 0.8375493375908025
Root Mean Squared Error: 0.9151772164945993
Mean Absolute Error: 0.7525158335209493

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
For layer 3
Mean Squared Error: 0.9405697698566822
Root Mean Squared Error: 0.9698297633382273
Mean Absolute Error: 0.7885089144316629

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
For layer 4
Mean Squared Error: 1.0382587857176275
Root Mean Squared Error: 1.018949844554494
Mean Absolute Error: 0.8264233215955279

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
For layer 5
Mean Squared Error: 0.9861151688744426
Root Mean Squared Error: 0.9930333171019201
Mean Absolute Error: 0.8052553303631838

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
For layer 6
Mean Squared Error: 0.978239744575152
Root Mean Squared Error: 0.9890600308247988
Mean Absolu

## Classification

We now analyze overfitting on a classification problem using the MNIST dataset.

In [43]:
(x_minst_train, y_minst_train), (x_minst_test, y_minst_test) = mnist.load_data()
assert x_minst_train.shape == (60000, 28, 28)
assert x_minst_test.shape == (10000, 28, 28)
assert y_minst_train.shape == (60000,)
assert y_minst_test.shape == (10000,)

We then searched for optimal neural network for MNIST on the internet 

links:

https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/

https://medium.com/@martin_stoyanov/optimizing-hyperparameters-for-the-mnist-dataset-in-javascript-4cb8c17df940


https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/multi-class_classification_with_MNIST.ipynb?hl=en#scrollTo=wYG5qXpP5a9n   # This is the original notebook READMEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE


https://www.tensorflow.org/datasets/keras_example # README TOO


https://www.kaggle.com/code/prashant111/mnist-deep-neural-network-with-keras # AND ME